# Union

https://www.kaggle.com/datasets/willianoliveiragibin/grocery-inventory

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("myApp").master("local[*]").getOrCreate()

In [2]:
#!/bin/bash
! curl -L -o inventory.zip https://www.kaggle.com/api/v1/datasets/download/willianoliveiragibin/grocery-inventory
! unzip inventory.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 50801  100 50801    0     0   102k      0 --:--:-- --:--:-- --:--:--  102k
Archive:  inventory.zip
  inflating: Grocery_Inventory new v1.csv  


In [3]:
inv = spark.read.csv("Grocery_Inventory new v1.csv", header=True, inferSchema=True)
inv.printSchema()

root
 |-- Product_Name: string (nullable = true)
 |-- Catagory: string (nullable = true)
 |-- Supplier_Name: string (nullable = true)
 |-- Warehouse_Location: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Supplier_ID: string (nullable = true)
 |-- Date_Received: string (nullable = true)
 |-- Last_Order_Date: string (nullable = true)
 |-- Expiration_Date: string (nullable = true)
 |-- Stock_Quantity: integer (nullable = true)
 |-- Reorder_Level: integer (nullable = true)
 |-- Reorder_Quantity: integer (nullable = true)
 |-- Unit_Price: string (nullable = true)
 |-- Sales_Volume: integer (nullable = true)
 |-- Inventory_Turnover_Rate: integer (nullable = true)
 |-- percentage: string (nullable = true)



In [4]:
inv.show(3)

+---------------+-------------------+-------------+--------------------+------------+-----------+-----------+-------------+---------------+---------------+--------------+-------------+----------------+----------+------------+-----------------------+----------+
|   Product_Name|           Catagory|Supplier_Name|  Warehouse_Location|      Status| Product_ID|Supplier_ID|Date_Received|Last_Order_Date|Expiration_Date|Stock_Quantity|Reorder_Level|Reorder_Quantity|Unit_Price|Sales_Volume|Inventory_Turnover_Rate|percentage|
+---------------+-------------------+-------------+--------------------+------------+-----------+-----------+-------------+---------------+---------------+--------------+-------------+----------------+----------+------------+-----------------------+----------+
|    Bell Pepper|Fruits & Vegetables|       Eimbee|20 Pennsylvania P...|Discontinued|29-017-6255|43-348-2450|     3/1/2024|       1/6/2025|      1/31/2025|            46|           64|              17|     $4.60|     

In [5]:
from pyspark.sql.functions import col

inv = inv.select(col("Catagory").alias("Category"), "Product_Name", "Date_Received", "Last_Order_Date", "Unit_Price")

In [6]:
inv.select("Category").distinct().show()

+-------------------+
|           Category|
+-------------------+
|             Bakery|
|        Oils & Fats|
|    Grains & Pulses|
|          Beverages|
|Fruits & Vegetables|
|              Dairy|
|            Seafood|
|               NULL|
+-------------------+



In [7]:
bakery = inv.filter(inv.Category == "Bakery")
bakery.count()

74

In [8]:
sea = inv.filter(inv.Category == "Seafood")
sea.count()

90

In [9]:
nulls = inv.filter(inv.Category.isNull())
nulls.show()

+--------+------------+-------------+---------------+----------+
|Category|Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+--------+------------+-------------+---------------+----------+
|    NULL|     Cabbage|   12/23/2024|     11/26/2024|    $66.55|
+--------+------------+-------------+---------------+----------+



In [10]:
bakery_nulls = bakery.union(nulls)
bakery_nulls.count()

75

In [11]:
bakery_nulls.sort("Category").show(3)

+--------+-----------------+-------------+---------------+----------+
|Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+--------+-----------------+-------------+---------------+----------+
|    NULL|          Cabbage|   12/23/2024|     11/26/2024|    $66.55|
|  Bakery|Digestive Biscuit|     9/1/2024|      3/17/2024|     $4.00|
|  Bakery|  Sourdough Bread|     4/2/2024|      6/10/2024|     $4.00|
+--------+-----------------+-------------+---------------+----------+
only showing top 3 rows



In [12]:
sea_nulls = sea.union(nulls)
sea_nulls.count()

91

In [13]:
sea_nulls.sort("Category").show(3)

+--------+------------+-------------+---------------+----------+
|Category|Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+--------+------------+-------------+---------------+----------+
|    NULL|     Cabbage|   12/23/2024|     11/26/2024|    $66.55|
| Seafood|   Anchovies|     6/1/2024|      2/23/2025|    $10.00|
| Seafood|     Tilapia|     5/2/2024|       1/6/2025|     $7.00|
+--------+------------+-------------+---------------+----------+
only showing top 3 rows



In [14]:
bakery_nulls.union(sea_nulls).count()

166

In [15]:
bakery_nulls.union(sea_nulls).sort("Category").show(3)

+--------+-----------------+-------------+---------------+----------+
|Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+--------+-----------------+-------------+---------------+----------+
|    NULL|          Cabbage|   12/23/2024|     11/26/2024|    $66.55|
|    NULL|          Cabbage|   12/23/2024|     11/26/2024|    $66.55|
|  Bakery|Digestive Biscuit|     9/1/2024|      3/17/2024|     $4.00|
+--------+-----------------+-------------+---------------+----------+
only showing top 3 rows



In [16]:
bakery_nulls = bakery_nulls.fillna(subset="Category", value="UNKNOWN")

In [17]:
sea_nulls = sea_nulls.fillna(subset="Category", value="UNKNOWN")

In [18]:
bakery_nulls.union(sea_nulls).count()

166

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.union.html#pyspark.sql.DataFrame.union

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.unionAll.html#pyspark.sql.DataFrame.unionAll

In [19]:
bakery_nulls.union(sea_nulls).distinct().count()

165

Problem with order of columns:

In [20]:
nulls = nulls.select("Category", "Unit_Price", "Product_Name", "Date_Received", "Last_Order_Date")
nulls.show()

+--------+----------+------------+-------------+---------------+
|Category|Unit_Price|Product_Name|Date_Received|Last_Order_Date|
+--------+----------+------------+-------------+---------------+
|    NULL|    $66.55|     Cabbage|   12/23/2024|     11/26/2024|
+--------+----------+------------+-------------+---------------+



In [21]:
bakery.show(3)

+--------+-----------------+-------------+---------------+----------+
|Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+--------+-----------------+-------------+---------------+----------+
|  Bakery|Digestive Biscuit|     9/1/2024|      3/17/2024|     $4.00|
|  Bakery|  Sourdough Bread|     4/2/2024|      6/10/2024|     $4.00|
|  Bakery|  Sourdough Bread|     7/2/2024|      3/19/2024|     $4.50|
+--------+-----------------+-------------+---------------+----------+
only showing top 3 rows



In [22]:
bakery.union(nulls).sort("Category").show(3)

+--------+-----------------+-------------+---------------+----------+
|Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+--------+-----------------+-------------+---------------+----------+
|    NULL|           $66.55|      Cabbage|     12/23/2024|11/26/2024|
|  Bakery|Digestive Biscuit|     9/1/2024|      3/17/2024|     $4.00|
|  Bakery|  Sourdough Bread|     4/2/2024|      6/10/2024|     $4.00|
+--------+-----------------+-------------+---------------+----------+
only showing top 3 rows



In [23]:
bakery.unionByName(nulls).sort("Category").show(3)

+--------+-----------------+-------------+---------------+----------+
|Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+--------+-----------------+-------------+---------------+----------+
|    NULL|          Cabbage|   12/23/2024|     11/26/2024|    $66.55|
|  Bakery|Digestive Biscuit|     9/1/2024|      3/17/2024|     $4.00|
|  Bakery|  Sourdough Bread|     4/2/2024|      6/10/2024|     $4.00|
+--------+-----------------+-------------+---------------+----------+
only showing top 3 rows



# sort & orderBy &  groupBy

In [24]:
bakery_nulls.show(10)

+--------+-----------------+-------------+---------------+----------+
|Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+--------+-----------------+-------------+---------------+----------+
|  Bakery|Digestive Biscuit|     9/1/2024|      3/17/2024|     $4.00|
|  Bakery|  Sourdough Bread|     4/2/2024|      6/10/2024|     $4.00|
|  Bakery|  Sourdough Bread|     7/2/2024|      3/19/2024|     $4.50|
|  Bakery|Whole Wheat Bread|     8/2/2024|      8/24/2024|     $3.50|
|  Bakery|Chocolate Biscuit|     8/2/2024|      3/21/2024|     $5.00|
|  Bakery|  Sourdough Bread|     6/3/2024|      1/16/2025|     $4.50|
|  Bakery|  Sourdough Bread|    11/3/2024|      6/10/2024|     $4.50|
|  Bakery|Chocolate Biscuit|    11/4/2024|       4/4/2024|     $5.00|
|  Bakery|Chocolate Biscuit|     3/5/2024|      9/16/2024|     $5.00|
|  Bakery|Chocolate Biscuit|     6/5/2024|      1/16/2025|     $5.00|
+--------+-----------------+-------------+---------------+----------+
only showing top 10 

In [25]:
from pyspark.sql.functions import desc, asc
bakery_nulls.orderBy(desc("Category")).show(10)

+--------+-----------------+-------------+---------------+----------+
|Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+--------+-----------------+-------------+---------------+----------+
| UNKNOWN|          Cabbage|   12/23/2024|     11/26/2024|    $66.55|
|  Bakery|Digestive Biscuit|     9/1/2024|      3/17/2024|     $4.00|
|  Bakery|  Sourdough Bread|     4/2/2024|      6/10/2024|     $4.00|
|  Bakery|  Sourdough Bread|     7/2/2024|      3/19/2024|     $4.50|
|  Bakery|Whole Wheat Bread|     8/2/2024|      8/24/2024|     $3.50|
|  Bakery|Chocolate Biscuit|     8/2/2024|      3/21/2024|     $5.00|
|  Bakery|  Sourdough Bread|     6/3/2024|      1/16/2025|     $4.50|
|  Bakery|  Sourdough Bread|    11/3/2024|      6/10/2024|     $4.50|
|  Bakery|Chocolate Biscuit|    11/4/2024|       4/4/2024|     $5.00|
|  Bakery|Chocolate Biscuit|     3/5/2024|      9/16/2024|     $5.00|
+--------+-----------------+-------------+---------------+----------+
only showing top 10 

In [26]:
bakery_nulls.orderBy(desc("Category"), asc("Product_Name")).show(10)

+--------+-----------------+-------------+---------------+----------+
|Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+--------+-----------------+-------------+---------------+----------+
| UNKNOWN|          Cabbage|   12/23/2024|     11/26/2024|    $66.55|
|  Bakery|   Butter Biscuit|     2/5/2025|       5/2/2024|     $6.00|
|  Bakery|   Butter Biscuit|    6/16/2024|      5/24/2024|     $6.00|
|  Bakery|   Butter Biscuit|     2/6/2025|      2/17/2025|     $6.00|
|  Bakery|   Butter Biscuit|   12/10/2024|      7/15/2024|     $6.00|
|  Bakery|   Butter Biscuit|    4/17/2024|       2/4/2025|     $6.00|
|  Bakery|   Butter Biscuit|    7/19/2024|      7/21/2024|     $6.00|
|  Bakery|Chocolate Biscuit|     1/7/2025|      9/19/2024|     $5.00|
|  Bakery|Chocolate Biscuit|   11/13/2024|      9/27/2024|     $5.00|
|  Bakery|Chocolate Biscuit|    10/6/2024|      1/14/2025|     $5.00|
+--------+-----------------+-------------+---------------+----------+
only showing top 10 

In [27]:
bakery_nulls.sort(desc("Category"), asc("Product_Name")).show(10)

+--------+-----------------+-------------+---------------+----------+
|Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+--------+-----------------+-------------+---------------+----------+
| UNKNOWN|          Cabbage|   12/23/2024|     11/26/2024|    $66.55|
|  Bakery|   Butter Biscuit|     2/5/2025|       5/2/2024|     $6.00|
|  Bakery|   Butter Biscuit|    6/16/2024|      5/24/2024|     $6.00|
|  Bakery|   Butter Biscuit|     2/6/2025|      2/17/2025|     $6.00|
|  Bakery|   Butter Biscuit|   12/10/2024|      7/15/2024|     $6.00|
|  Bakery|   Butter Biscuit|    4/17/2024|       2/4/2025|     $6.00|
|  Bakery|   Butter Biscuit|    7/19/2024|      7/21/2024|     $6.00|
|  Bakery|Chocolate Biscuit|     1/7/2025|      9/19/2024|     $5.00|
|  Bakery|Chocolate Biscuit|   11/13/2024|      9/27/2024|     $5.00|
|  Bakery|Chocolate Biscuit|    10/6/2024|      1/14/2025|     $5.00|
+--------+-----------------+-------------+---------------+----------+
only showing top 10 

## groupBy

In [28]:
inv.groupBy("Category").count().orderBy(asc("Category")).show()

+-------------------+-----+
|           Category|count|
+-------------------+-----+
|               NULL|    1|
|             Bakery|   74|
|          Beverages|   75|
|              Dairy|  180|
|Fruits & Vegetables|  331|
|    Grains & Pulses|  162|
|        Oils & Fats|   77|
|            Seafood|   90|
+-------------------+-----+



In [29]:
from pyspark.sql.functions import count

inv.groupBy("Category").agg(count("Product_Name")).orderBy(asc("Category")).show()

+-------------------+-------------------+
|           Category|count(Product_Name)|
+-------------------+-------------------+
|               NULL|                  1|
|             Bakery|                 74|
|          Beverages|                 75|
|              Dairy|                180|
|Fruits & Vegetables|                331|
|    Grains & Pulses|                162|
|        Oils & Fats|                 77|
|            Seafood|                 90|
+-------------------+-------------------+



In [30]:
inv\
  .groupBy("Category")\
  .agg(count("Product_Name").alias("NumberOfRecords"))\
  .orderBy(asc("Category"))\
  .show()

+-------------------+---------------+
|           Category|NumberOfRecords|
+-------------------+---------------+
|               NULL|              1|
|             Bakery|             74|
|          Beverages|             75|
|              Dairy|            180|
|Fruits & Vegetables|            331|
|    Grains & Pulses|            162|
|        Oils & Fats|             77|
|            Seafood|             90|
+-------------------+---------------+



In [31]:
inv\
  .groupBy("Category")\
  .agg(count("Product_Name"))\
  .withColumn("NumberOfRecords", col("count(Product_Name)"))\
  .drop("count(Product_Name)")\
  .orderBy(asc("Category"))\
  .show()

+-------------------+---------------+
|           Category|NumberOfRecords|
+-------------------+---------------+
|               NULL|              1|
|             Bakery|             74|
|          Beverages|             75|
|              Dairy|            180|
|Fruits & Vegetables|            331|
|    Grains & Pulses|            162|
|        Oils & Fats|             77|
|            Seafood|             90|
+-------------------+---------------+



### Basic Aggregation Functions:
**sum()** - Calculates the sum of values in a column.

**avg() or mean()** - Computes the arithmetic average of values in a column.

**count()** - Counts the number of rows (or values in a column).

**min()** - Finds the minimum value in a column.

**max()** - Finds the maximum value in a column.

### Advanced Aggregation Functions:
**stddev()** - Calculates the standard deviation.

**variance()** - Computes the variance.

**approx_count_distinct()** - Estimates the number of distinct values in a column.

**first()** - Returns the first non-NULL row.

**last()** - Returns the last non-NULL row.

**collect_list()** - Collects values from a column into a list.

**collect_set()** - Collects distinct values from a column into a set.

**skewness()** - Measures the skewness of a column.

**kurtosis()** - Calculates the kurtosis of a column.

### Time-Related Aggregation Functions:
**grouping()** - Determines whether a row belongs to a grouping set in group aggregations.

**window()** - Enables performing aggregation operations over time windows.

In [32]:
from pyspark.sql.functions import count, sum, avg, min, max

In [33]:
inv\
  .groupBy("Category")\
  .agg(count("Product_Name").alias("Count"),
       sum("Unit_Price").alias("Sum"),
       avg("Unit_Price").alias("Avg"),
       min("Unit_Price").alias("Min"),
       max("Unit_Price").alias("Max")
       )\
  .orderBy(asc("Category"))\
  .show()

+-------------------+-----+----+----+------+------+
|           Category|Count| Sum| Avg|   Min|   Max|
+-------------------+-----+----+----+------+------+
|               NULL|    1|NULL|NULL|$66.55|$66.55|
|             Bakery|   74|NULL|NULL| $2.50| $6.00|
|          Beverages|   75|NULL|NULL|$10.00| $8.50|
|              Dairy|  180|NULL|NULL| $0.20| $9.50|
|Fruits & Vegetables|  331|NULL|NULL| $0.90|$98.43|
|    Grains & Pulses|  162|NULL|NULL| $1.45| $9.75|
|        Oils & Fats|   77|NULL|NULL| $1.80| $6.50|
|            Seafood|   90|NULL|NULL|$10.00| $9.00|
+-------------------+-----+----+----+------+------+



In [34]:
inv.printSchema()

root
 |-- Category: string (nullable = true)
 |-- Product_Name: string (nullable = true)
 |-- Date_Received: string (nullable = true)
 |-- Last_Order_Date: string (nullable = true)
 |-- Unit_Price: string (nullable = true)



In [35]:
from pyspark.sql.functions import col, to_date, regexp_replace

inv = inv\
  .withColumn("Date_Received", to_date(col("Date_Received"),"M/d/yyyy"))\
  .withColumn("Last_Order_Date", to_date(col("Last_Order_date"),"M/d/yyyy"))\
  .withColumn("Unit_Price", regexp_replace(col("Unit_Price"), r"\$", "").cast("double"))

In [36]:
inv.printSchema()

root
 |-- Category: string (nullable = true)
 |-- Product_Name: string (nullable = true)
 |-- Date_Received: date (nullable = true)
 |-- Last_Order_Date: date (nullable = true)
 |-- Unit_Price: double (nullable = true)



In [37]:
inv\
  .groupBy("Category")\
  .agg(count("Product_Name").alias("Count"),
       sum("Unit_Price").alias("Sum"),
       avg("Unit_Price").alias("Avg"),
       min("Unit_Price").alias("Min"),
       max("Unit_Price").alias("Max")
       )\
  .orderBy(asc("Category"))\
  .show()

+-------------------+-----+------------------+------------------+-----+-----+
|           Category|Count|               Sum|               Avg|  Min|  Max|
+-------------------+-----+------------------+------------------+-----+-----+
|               NULL|    1|             66.55|             66.55|66.55|66.55|
|             Bakery|   74|300.70000000000005| 4.063513513513514|  2.5|  6.0|
|          Beverages|   75|           1199.85|            15.998|  4.7| 31.0|
|              Dairy|  180| 811.5000000000001| 4.508333333333334|  0.2| 12.0|
|Fruits & Vegetables|  331|1525.6999999999998| 4.609365558912386|  0.9|98.43|
|    Grains & Pulses|  162| 623.4499999999999|3.8484567901234565| 1.45| 10.0|
|        Oils & Fats|   77| 326.6000000000001| 4.241558441558443|  1.8| 10.0|
|            Seafood|   90|            1010.6| 11.22888888888889|  6.0| 20.0|
+-------------------+-----+------------------+------------------+-----+-----+



In [38]:
from pyspark.sql.functions import round

inv\
  .groupBy("Category")\
  .agg(count("Product_Name").alias("Count"),
       round(sum("Unit_Price"),2).alias("Sum"),
       round(avg("Unit_Price"),2).alias("Avg"),
       min("Unit_Price").alias("Min"),
       max("Unit_Price").alias("Max")
       )\
  .orderBy(asc("Category"))\
  .show()

+-------------------+-----+-------+-----+-----+-----+
|           Category|Count|    Sum|  Avg|  Min|  Max|
+-------------------+-----+-------+-----+-----+-----+
|               NULL|    1|  66.55|66.55|66.55|66.55|
|             Bakery|   74|  300.7| 4.06|  2.5|  6.0|
|          Beverages|   75|1199.85| 16.0|  4.7| 31.0|
|              Dairy|  180|  811.5| 4.51|  0.2| 12.0|
|Fruits & Vegetables|  331| 1525.7| 4.61|  0.9|98.43|
|    Grains & Pulses|  162| 623.45| 3.85| 1.45| 10.0|
|        Oils & Fats|   77|  326.6| 4.24|  1.8| 10.0|
|            Seafood|   90| 1010.6|11.23|  6.0| 20.0|
+-------------------+-----+-------+-----+-----+-----+



In [39]:
df_grouped = inv\
  .groupBy("Category")\
  .agg(count("Product_Name").alias("Count"),
       round(sum("Unit_Price"),2).alias("Sum"),
       round(avg("Unit_Price"),2).alias("Avg"),
       min("Unit_Price").alias("Min"),
       max("Unit_Price").alias("Max")
       )\
   .orderBy(asc("Category"))

In [40]:
df_grouped.show()

+-------------------+-----+-------+-----+-----+-----+
|           Category|Count|    Sum|  Avg|  Min|  Max|
+-------------------+-----+-------+-----+-----+-----+
|               NULL|    1|  66.55|66.55|66.55|66.55|
|             Bakery|   74|  300.7| 4.06|  2.5|  6.0|
|          Beverages|   75|1199.85| 16.0|  4.7| 31.0|
|              Dairy|  180|  811.5| 4.51|  0.2| 12.0|
|Fruits & Vegetables|  331| 1525.7| 4.61|  0.9|98.43|
|    Grains & Pulses|  162| 623.45| 3.85| 1.45| 10.0|
|        Oils & Fats|   77|  326.6| 4.24|  1.8| 10.0|
|            Seafood|   90| 1010.6|11.23|  6.0| 20.0|
+-------------------+-----+-------+-----+-----+-----+



In [41]:
df_grouped\
  .where(col("Avg") > 10)\
  .show()

+---------+-----+-------+-----+-----+-----+
| Category|Count|    Sum|  Avg|  Min|  Max|
+---------+-----+-------+-----+-----+-----+
|     NULL|    1|  66.55|66.55|66.55|66.55|
|Beverages|   75|1199.85| 16.0|  4.7| 31.0|
|  Seafood|   90| 1010.6|11.23|  6.0| 20.0|
+---------+-----+-------+-----+-----+-----+



In [42]:
df_grouped\
  .where("Avg > 10")\
  .show()

+---------+-----+-------+-----+-----+-----+
| Category|Count|    Sum|  Avg|  Min|  Max|
+---------+-----+-------+-----+-----+-----+
|     NULL|    1|  66.55|66.55|66.55|66.55|
|Beverages|   75|1199.85| 16.0|  4.7| 31.0|
|  Seafood|   90| 1010.6|11.23|  6.0| 20.0|
+---------+-----+-------+-----+-----+-----+



# Unique values (distinct) and how to use it

In [43]:
inv.show()

+-------------------+-----------------+-------------+---------------+----------+
|           Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+-------------------+-----------------+-------------+---------------+----------+
|Fruits & Vegetables|      Bell Pepper|   2024-03-01|     2025-01-06|       4.6|
|        Oils & Fats|    Vegetable Oil|   2024-04-01|     2024-05-19|       2.0|
|              Dairy|  Parmesan Cheese|   2024-04-01|     2024-12-21|      12.0|
|Fruits & Vegetables|           Carrot|   2024-05-01|     2024-12-12|       1.5|
|Fruits & Vegetables|           Garlic|   2024-05-01|     2024-07-28|       7.0|
|Fruits & Vegetables|            Lemon|   2024-05-01|     2024-08-07|       2.4|
|    Grains & Pulses|    Coconut Sugar|   2024-05-01|     2025-01-29|       5.0|
|            Seafood|        Anchovies|   2024-06-01|     2025-02-23|      10.0|
|              Dairy|           Cheese|   2024-06-01|     2024-06-03|       9.0|
|              Dairy|       

In [44]:
categories = inv.select("Category")

In [45]:
categories.show()

+-------------------+
|           Category|
+-------------------+
|Fruits & Vegetables|
|        Oils & Fats|
|              Dairy|
|Fruits & Vegetables|
|Fruits & Vegetables|
|Fruits & Vegetables|
|    Grains & Pulses|
|            Seafood|
|              Dairy|
|              Dairy|
|              Dairy|
|        Oils & Fats|
|Fruits & Vegetables|
|             Bakery|
|Fruits & Vegetables|
|Fruits & Vegetables|
|              Dairy|
|              Dairy|
|Fruits & Vegetables|
|    Grains & Pulses|
+-------------------+
only showing top 20 rows



In [46]:
categories.distinct().show()

+-------------------+
|           Category|
+-------------------+
|             Bakery|
|        Oils & Fats|
|    Grains & Pulses|
|          Beverages|
|Fruits & Vegetables|
|              Dairy|
|            Seafood|
|               NULL|
+-------------------+



In [47]:
inv.filter(col("Category").isNotNull()).groupBy("Category").count().orderBy(col("count")).limit(3).show()

+-----------+-----+
|   Category|count|
+-----------+-----+
|     Bakery|   74|
|  Beverages|   75|
|Oils & Fats|   77|
+-----------+-----+



In [48]:
result = inv.filter(col("Category").isNotNull()).groupBy("Category").count().orderBy(col("count")).limit(3).select("Category").collect()
selected_categories = [row.Category for row in result]
selected_categories

['Bakery', 'Beverages', 'Oils & Fats']

In [49]:
inv_select = inv.filter(col("Category").isin(selected_categories))
inv_select.show(inv_select.count())

+-----------+-----------------+-------------+---------------+----------+
|   Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|
+-----------+-----------------+-------------+---------------+----------+
|Oils & Fats|    Vegetable Oil|   2024-04-01|     2024-05-19|       2.0|
|Oils & Fats|      Avocado Oil|   2024-07-01|     2024-04-06|      10.0|
|     Bakery|Digestive Biscuit|   2024-09-01|     2024-03-17|       4.0|
|  Beverages|   Arabica Coffee|   2024-11-01|     2024-05-29|      20.0|
|     Bakery|  Sourdough Bread|   2024-04-02|     2024-06-10|       4.0|
|  Beverages|     Green Coffee|   2024-04-02|     2024-11-26|      12.0|
|  Beverages|   Arabica Coffee|   2024-04-02|     2025-01-07|      20.0|
|     Bakery|  Sourdough Bread|   2024-07-02|     2024-03-19|       4.5|
|     Bakery|Whole Wheat Bread|   2024-08-02|     2024-08-24|       3.5|
|  Beverages|   Robusta Coffee|   2024-08-02|     2024-08-28|      10.0|
|     Bakery|Chocolate Biscuit|   2024-08-02|     2

# Window functions

In [50]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, min, max, avg,  sum, col, desc

In [51]:
window_spec = Window.partitionBy("Category").orderBy("date_received")

row_num_func = row_number().over(window_spec)
min_func = min(col("Unit_Price")).over(window_spec)
max_func = max(col("Unit_Price")).over(window_spec)
avg_func = avg(col("Unit_Price")).over(window_spec)
sum_func = sum(col("Unit_Price")).over(window_spec)


In [52]:
inv_select\
  .withColumn("row_num_per_category", row_num_func)\
  .withColumn("min_per_cat", min_func)\
  .withColumn("max_per_cat", max_func)\
  .withColumn("avg_per_cat", avg_func)\
  .withColumn("sum_per_cat", sum_func)\
  .show(inv_select.count())

+-----------+-----------------+-------------+---------------+----------+--------------------+-----------+-----------+------------------+------------------+
|   Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|row_num_per_category|min_per_cat|max_per_cat|       avg_per_cat|       sum_per_cat|
+-----------+-----------------+-------------+---------------+----------+--------------------+-----------+-----------+------------------+------------------+
|     Bakery|Chocolate Biscuit|   2024-03-05|     2024-09-16|       5.0|                   1|        5.0|        5.0|               5.0|               5.0|
|     Bakery|  Oatmeal Biscuit|   2024-03-07|     2025-01-01|       5.0|                   2|        5.0|        5.0|               5.0|              10.0|
|     Bakery|Digestive Biscuit|   2024-03-09|     2024-11-24|       4.0|                   3|        4.0|        5.0| 4.666666666666667|              14.0|
|     Bakery|  Vanilla Biscuit|   2024-03-18|     2024-03-20|   

In [53]:
window_spec = Window.partitionBy("Category").orderBy(desc("unit_price"))
row_num_func = row_number().over(window_spec)

In [54]:
inv_select\
  .withColumn("row_num_per_cat", row_num_func)\
  .filter(col("row_num_per_cat") <= 3)\
  .show(inv_select.count())

+-----------+--------------+-------------+---------------+----------+---------------+
|   Category|  Product_Name|Date_Received|Last_Order_Date|Unit_Price|row_num_per_cat|
+-----------+--------------+-------------+---------------+----------+---------------+
|     Bakery|Butter Biscuit|   2024-12-10|     2024-07-15|       6.0|              1|
|     Bakery|Butter Biscuit|   2025-02-05|     2024-05-02|       6.0|              2|
|     Bakery|Butter Biscuit|   2025-02-06|     2025-02-17|       6.0|              3|
|  Beverages|    Herbal Tea|   2025-01-28|     2024-10-05|      31.0|              1|
|  Beverages|    Herbal Tea|   2024-06-26|     2025-02-22|      30.5|              2|
|  Beverages|    Herbal Tea|   2024-11-05|     2024-08-19|      30.0|              3|
|Oils & Fats|   Avocado Oil|   2024-07-01|     2024-04-06|      10.0|              1|
|Oils & Fats|   Avocado Oil|   2024-10-04|     2025-01-25|      10.0|              2|
|Oils & Fats|   Avocado Oil|   2024-04-10|     2024-10

In [55]:
from pyspark.sql.functions import dense_rank

dense_rank_func = dense_rank().over(window_spec)

inv_select\
  .withColumn("dense_rank_per_cat", dense_rank_func)\
  .show(inv_select.count())

+-----------+-----------------+-------------+---------------+----------+------------------+
|   Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|dense_rank_per_cat|
+-----------+-----------------+-------------+---------------+----------+------------------+
|     Bakery|   Butter Biscuit|   2024-12-10|     2024-07-15|       6.0|                 1|
|     Bakery|   Butter Biscuit|   2025-02-05|     2024-05-02|       6.0|                 1|
|     Bakery|   Butter Biscuit|   2025-02-06|     2025-02-17|       6.0|                 1|
|     Bakery|   Butter Biscuit|   2024-04-17|     2025-02-04|       6.0|                 1|
|     Bakery|   Butter Biscuit|   2024-06-16|     2024-05-24|       6.0|                 1|
|     Bakery|   Butter Biscuit|   2024-07-19|     2024-07-21|       6.0|                 1|
|     Bakery|  Vanilla Biscuit|   2024-05-12|     2024-06-25|       5.5|                 2|
|     Bakery|  Vanilla Biscuit|   2024-03-18|     2024-03-20|       5.5|        

In [56]:
inv_select\
  .withColumn("dense_rank_per_cat", dense_rank_func)\
  .filter(col("dense_rank_per_cat") <= 3)\
  .show(inv_select.count())

+-----------+-----------------+-------------+---------------+----------+------------------+
|   Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|dense_rank_per_cat|
+-----------+-----------------+-------------+---------------+----------+------------------+
|     Bakery|   Butter Biscuit|   2024-12-10|     2024-07-15|       6.0|                 1|
|     Bakery|   Butter Biscuit|   2025-02-05|     2024-05-02|       6.0|                 1|
|     Bakery|   Butter Biscuit|   2025-02-06|     2025-02-17|       6.0|                 1|
|     Bakery|   Butter Biscuit|   2024-04-17|     2025-02-04|       6.0|                 1|
|     Bakery|   Butter Biscuit|   2024-06-16|     2024-05-24|       6.0|                 1|
|     Bakery|   Butter Biscuit|   2024-07-19|     2024-07-21|       6.0|                 1|
|     Bakery|  Vanilla Biscuit|   2024-05-12|     2024-06-25|       5.5|                 2|
|     Bakery|  Vanilla Biscuit|   2024-03-18|     2024-03-20|       5.5|        

In [57]:
from pyspark.sql.functions import rank

rank_func = rank().over(window_spec)

inv_select\
  .withColumn("rank_per_cat", rank_func)\
  .show(inv_select.count())

+-----------+-----------------+-------------+---------------+----------+------------+
|   Category|     Product_Name|Date_Received|Last_Order_Date|Unit_Price|rank_per_cat|
+-----------+-----------------+-------------+---------------+----------+------------+
|     Bakery|   Butter Biscuit|   2024-12-10|     2024-07-15|       6.0|           1|
|     Bakery|   Butter Biscuit|   2025-02-05|     2024-05-02|       6.0|           1|
|     Bakery|   Butter Biscuit|   2025-02-06|     2025-02-17|       6.0|           1|
|     Bakery|   Butter Biscuit|   2024-04-17|     2025-02-04|       6.0|           1|
|     Bakery|   Butter Biscuit|   2024-06-16|     2024-05-24|       6.0|           1|
|     Bakery|   Butter Biscuit|   2024-07-19|     2024-07-21|       6.0|           1|
|     Bakery|  Vanilla Biscuit|   2024-05-12|     2024-06-25|       5.5|           7|
|     Bakery|  Vanilla Biscuit|   2024-03-18|     2024-03-20|       5.5|           7|
|     Bakery|  Vanilla Biscuit|   2024-07-27|     2024

In [58]:
inv_select\
  .withColumn("rank_per_cat", rank_func)\
  .filter(col("rank_per_cat") <= 3)\
  .show(inv_select.count())

+-----------+--------------+-------------+---------------+----------+------------+
|   Category|  Product_Name|Date_Received|Last_Order_Date|Unit_Price|rank_per_cat|
+-----------+--------------+-------------+---------------+----------+------------+
|     Bakery|Butter Biscuit|   2024-12-10|     2024-07-15|       6.0|           1|
|     Bakery|Butter Biscuit|   2025-02-05|     2024-05-02|       6.0|           1|
|     Bakery|Butter Biscuit|   2025-02-06|     2025-02-17|       6.0|           1|
|     Bakery|Butter Biscuit|   2024-04-17|     2025-02-04|       6.0|           1|
|     Bakery|Butter Biscuit|   2024-06-16|     2024-05-24|       6.0|           1|
|     Bakery|Butter Biscuit|   2024-07-19|     2024-07-21|       6.0|           1|
|  Beverages|    Herbal Tea|   2025-01-28|     2024-10-05|      31.0|           1|
|  Beverages|    Herbal Tea|   2024-06-26|     2025-02-22|      30.5|           2|
|  Beverages|    Herbal Tea|   2024-11-05|     2024-08-19|      30.0|           3|
|  B

In [59]:
inv_select\
  .withColumn("rank_per_cat", rank().over(Window.partitionBy("Category").orderBy(desc("unit_price"))))\
  .filter(col("rank_per_cat") <= 3)\
  .show(inv_select.count())

+-----------+--------------+-------------+---------------+----------+------------+
|   Category|  Product_Name|Date_Received|Last_Order_Date|Unit_Price|rank_per_cat|
+-----------+--------------+-------------+---------------+----------+------------+
|     Bakery|Butter Biscuit|   2024-12-10|     2024-07-15|       6.0|           1|
|     Bakery|Butter Biscuit|   2025-02-05|     2024-05-02|       6.0|           1|
|     Bakery|Butter Biscuit|   2025-02-06|     2025-02-17|       6.0|           1|
|     Bakery|Butter Biscuit|   2024-04-17|     2025-02-04|       6.0|           1|
|     Bakery|Butter Biscuit|   2024-06-16|     2024-05-24|       6.0|           1|
|     Bakery|Butter Biscuit|   2024-07-19|     2024-07-21|       6.0|           1|
|  Beverages|    Herbal Tea|   2025-01-28|     2024-10-05|      31.0|           1|
|  Beverages|    Herbal Tea|   2024-06-26|     2025-02-22|      30.5|           2|
|  Beverages|    Herbal Tea|   2024-11-05|     2024-08-19|      30.0|           3|
|  B

In [60]:
from pyspark.sql.functions import expr

inv_select\
  .withColumn("rank_per_cat", expr("rank() over(partition by category order by unit_price)"))\
  .filter(col("rank_per_cat") <= 3)\
  .show(inv_select.count())

+-----------+------------+-------------+---------------+----------+------------+
|   Category|Product_Name|Date_Received|Last_Order_Date|Unit_Price|rank_per_cat|
+-----------+------------+-------------+---------------+----------+------------+
|     Bakery| White Bread|   2024-07-11|     2024-11-18|       2.5|           1|
|     Bakery| White Bread|   2024-09-12|     2024-03-05|       2.5|           1|
|     Bakery| White Bread|   2024-10-19|     2024-06-14|       2.5|           1|
|     Bakery| White Bread|   2024-12-16|     2024-05-29|       2.5|           1|
|     Bakery| White Bread|   2024-03-21|     2024-02-29|       2.5|           1|
|     Bakery| White Bread|   2024-03-28|     2025-01-28|       2.5|           1|
|     Bakery| White Bread|   2024-05-25|     2024-07-28|       2.5|           1|
|     Bakery| White Bread|   2024-06-29|     2024-05-17|       2.5|           1|
|     Bakery| White Bread|   2024-08-22|     2024-09-03|       2.5|           1|
|  Beverages|   Black Tea|  